#### 0. Change directory

In [1]:
# Reference: https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script/15622021#15622021
# Here choosing the folder that stores all the modelling modules
import sys
sys.path.append(r'S:\\Grid_Orientation_code_006_machine_C002AU\\LISFLOOD_FP\\Modelling')

#### 1. Packages

In [2]:
from folder import *
from randomisation import random_values_generation
from dataPreparation import download_lidar, \
                            padding_combination, \
                            dem_raster_reference, \
                            terrain_shading             # For lidar downloading, padding calculation, dem reference generation, and terrain shading

from transformation import center_calculation, get_url, get_tile_files, \
                           MyPool, transformation_performance, transformation_parallelism # For transformation using multiprocessing

from rasterSimulation import raster_generation

from floodModel import tide_flow_data, bdy_generation, tideboundary_points, flood_simulation

# For creating plot
import matplotlib.pyplot

# For untransformation
from untransformation import untransformation_parallelism

import multiprocessing
from functools import partial


# For terrain shading
import os
import rioxarray as rxr
import xarray as xr
from osgeo import gdal

#### 2. Necessary variables

In [3]:
# # 20-m resolution
# # Construct a list of boundary
# resolution = 20
# number_pixel_x = 16 * 21
# number_pixel_y = 16 * 14

# xmin = 1767790
# ymin = 5471400

In [4]:
# 10-m resolution
# Construct a list of boundary
resolution = 10
number_pixel_x = 16 * 42
number_pixel_y = 16 * 28

xmin = 1767790
ymin = 5471400

In [5]:
# # 5-m resolution
# # Construct a list of boundary
# resolution = 5
# number_pixel_x = 16 * 84
# number_pixel_y = 16 * 56

# xmin = 1767790
# ymin = 5471400

In [6]:
# # 2-m resolution
# # Construct a list of boundary
# resolution = 2
# number_pixel_x = 16 * 210
# number_pixel_y = 16 * 140

# xmin = 1767790
# ymin = 5471400

##### 2.2. Other basic variables

In [7]:
xmax = xmin + resolution * number_pixel_x
ymax = ymin + resolution * number_pixel_y

# Boundary for the area of interest
boundary_1 = [xmin, ymin, xmax, ymax]

# Padding boundary for DEMs (use this boundary to create padding)
addition_2 = 16 * 3
boundary_2 = padding_combination(boundary_1, addition_2)

# Boundary for tiles (use this boundary to download LiDAR)
addition_3 = 16 * 3
boundary_3 = padding_combination(boundary_2, addition_3)

In [8]:
# LiDAR dataset name
lidar_name = "Wellington_2013"

# # Chunks and processors information (for computer has 8 cores)
# For 10-m resolution
size_of_chunk = 100
size_of_processor = 4
# # For 20-m resolution
# size_of_chunk = 50
# size_of_processor = 2
# # For 2-m resolution
# size_of_chunk = 400
# size_of_processor = 6
# # For 5-m resolution
# size_of_chunk = 200
# size_of_processor = 6

# Flow data
# flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_multiple_events\\flow_5yevent_001.csv"
# flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_multiple_events\\flow_10yevent_001.csv"
# flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_multiple_events\\flow_20yevent_001.csv"
# flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_multiple_events\\flow_50yevent_001.csv"
flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_multiple_events\\flow_1000yevent_001.csv"
# flow_path = r"P:\Martin_publication\Grid_Orientation\Flow_Tide\\flow.csv"

# Tide data
tide_path = r"P:\\Martin_publication\\Grid_Orientation\\Flow_Tide\\tide.csv"

# date start and end
date_start = r"2005-01-05 00:00:00"
date_end = r"2005-01-07 00:00:00"

# Tide flow data
tide_flow_data(tide_path, flow_path, date_start, date_end)

# Extracting flowdepth rate (for flowdepth_extraction() function)
flowdepth_rate = 0

# Time to extract water (for water_extraction() function)
extract_wd = "out.max"
extract_wse = "out.mxe"
extract_elev = "elev"

##### 2.3. Preparing flood model parameters

In [9]:
# Create tide_flow data
tide_flow_data(tide_path, flow_path, date_start, date_end)

# River points
river_points = (1774477.413, 5471521.558)

# Create BDY file
domain_name = "Waikanae"
bdy_generation(resolution, domain_name)

# Create tide boundary
tideboundary_points(boundary_1, False)

##### 2.4. Preparing some flood model inputs

In [10]:
# %%time
# # Download LiDAR
# download_lidar(boundary_3, lidar_name)

# # Reference DEM without padding
# dem_raster_reference(resolution, size_of_chunk, size_of_processor,
#                      boundary_1, lidar_name, "no_padding")

# # Reference DEM with padding
# dem_raster_reference(resolution, size_of_chunk, size_of_processor,
#                      boundary_2, lidar_name, "padding")

# # DEM for hillshade/terrain shade
# dem_raster_reference(2, 400, 6,
#                      boundary_1, lidar_name, "shading")

# # Terrain shading
# terrain_shading(50, 355)

# Calculate coordinates of center point
center_point = center_calculation(True)
center_x = center_point[0]                     # Extract x coordinate of center point
center_y = center_point[1]                     # Extract y coordinate of center point

# Get url list file
url_list_file = get_url(lidar_name)
tile_files = get_tile_files(lidar_name)

##### 2.5. Random transformations

##### 2.5.1. E translation (x)

In [11]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     50,
#     [0, 0],
#     [0, resolution/2],
#     [0, 0],
#     'uniform',
#     True
# )

##### 2.5.2. N translation (y)

In [12]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     50,
#     [0, 0],
#     [0, 0],
#     [0, resolution/2],
#     'uniform',
#     True
# )

##### 2.5.3. N-E translation

In [13]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     50,
#     [0, 0],
#     [0, resolution/2],
#     [0, resolution/2],
#     'uniform',
#     True
# )

##### 2.5.4. Rotation

In [14]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     50,
#     [0, 90],
#     [0, 0],
#     [0, 0],
#     'uniform',
#     True
# )

In [15]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     1,
#     [0, 0],
#     [0, 0],
#     [0, 0],
#     'uniform',
#     True
# )

In [16]:
# ran_trans

##### 2.5.5. N-E translation and rotation

In [17]:
# Transformation values
ran_trans = random_values_generation(
    1,
    50,
    [0, 90],
    [0, resolution/2],
    [0, resolution/2],
    'uniform',
    True
)

In [18]:
ran_trans

array([[4.60639462e+01, 4.75231848e+00, 7.20798064e-01],
       [8.53784502e+01, 1.55915726e+00, 2.11663224e+00],
       [7.44932334e+01, 2.04599568e+00, 2.74796844e+00],
       [2.48032019e+00, 3.76756554e+00, 2.69071657e+00],
       [2.96758545e+01, 3.94214352e+00, 1.51597415e+00],
       [4.08148101e+01, 6.70208486e-01, 2.01556493e+00],
       [1.83109717e+01, 1.31156670e+00, 3.75182336e+00],
       [2.52367882e+01, 2.42595487e+00, 4.90368600e+00],
       [8.65491474e+01, 3.62394970e+00, 2.70613428e+00],
       [2.49202084e+01, 8.03260044e-01, 4.84962707e+00],
       [4.64461727e+01, 5.79328062e-01, 3.11744878e+00],
       [6.99014803e+01, 3.06501651e+00, 4.58648852e+00],
       [3.56335890e+00, 2.64294632e+00, 2.29667941e+00],
       [5.61146212e+00, 3.20664085e+00, 4.26316419e+00],
       [5.33646916e+01, 1.30048724e+00, 4.19940761e+00],
       [4.58546293e+01, 2.55444442e+00, 3.76515104e+00],
       [1.33129832e+01, 4.09813360e+00, 3.41643453e+00],
       [7.08387247e+01, 9.58081

##### 2.6. Only for 2-m resolution (if not skip this part)

In [19]:
# # Transformation values
# ran_trans = random_values_generation(
#     1,
#     70,
#     [0, 90],
#     [0, resolution/2],
#     [0, resolution/2],
#     'uniform',
#     True
# )

In [20]:
# import numpy as np
# ran_trans = np.delete(ran_trans, [3, 4, 12, 40, 41], 0)
# ran_trans = ran_trans[:50]

In [21]:
# ran_trans

#### 3. Execution

In [22]:
# %%time
# transformation_parallelism(
#     center_x,
#     center_y,
#     lidar_name,
#     tile_files,
#     30,
#     url_list_file,
#     ran_trans,
#     1
# )

In [23]:
# %%time
# # DEM CREATION
# # List parameters
# resolution_func = resolution
# chunk_size_func = size_of_chunk
# processor_func = size_of_processor
# padding_func = boundary_2
# lidar_dataset_name = lidar_name

# # Design func
# func = partial(
#     raster_generation,
#     resolution_func,
#     chunk_size_func,
#     processor_func,
#     padding_func,
#     lidar_dataset_name,
#     center_x, center_y
# )

# # Design pool
# pool = MyPool(1)
# pool.map(func, [ran for ran in ran_trans])
# pool.close()
# pool.join()

In [24]:
# %%time
# for ran_trans_i in ran_trans:
#     flood_simulation(
#         domain_name,
#         river_points,
#         center_x, center_y,
#         fr"S:\\new_versions\\data",
#         ran_trans_i
#     )

In [25]:
%%time
# Water depth
untransformation_parallelism(
    extract_wd,
    ran_trans,
    30
)

CPU times: total: 62.5 ms
Wall time: 5min 14s


In [26]:
%%time
# Water surface elevation
untransformation_parallelism(
    extract_wse,
    ran_trans,
    30
)

CPU times: total: 78.1 ms
Wall time: 5min 21s


In [ ]:
# %%time
# # Elevation
# untransformation_parallelism(
#     extract_elev,
#     ran_trans,
#     1
# )